In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
df = pd.read_csv('clean_new.csv')
df.head()

,budget,genres,id,keywords,original_language,popularity,production_companies,production_countries,revenue,runtime,status,title,vote_average,vote_count,cast,year,director,producer
0,27000000,Action | Adventure | Science Fiction | Romance,79698,Action | Adventure | Science Fiction | Romance,en,2.418535,Corsan | Bliss Media | Limelight International...,Australia | Belgium | India,0,109.0,Released,The Lovers,4.8,34,Josh Hartnett | Simone Kessell,2015,Roland Joffé,Paul Breuls
1,15000000,Action | Adventure | Drama | Horror | Science ...,315011,Action | Adventure | Drama | Horror | Science ...,ja,9.476999,Cine Bazar | Toho Pictures,Japan,77000000,120.0,Released,Shin Godzilla,6.5,143,Hiroki Hasegawa | Yutaka Takenouchi,2016,Hideaki Anno,Kazutoshi Wadakura
2,70000000,Adventure | Action | Animation,2310,Adventure | Action | Animation,en,35.601665,Paramount Pictures | Shangri-La Entertainment ...,United States of America,195735876,115.0,Released,Beowulf,5.5,841,Ray Winstone | Angelina Jolie,2007,Robert Zemeckis,Robert Zemeckis
3,97250400,Fantasy | Adventure | Comedy | Family,2395,Fantasy | Adventure | Comedy | Family,fr,20.344364,Constantin Film | TF1 Films Productions | Path...,Belgium | France | Germany | Italy | Spain,132900000,116.0,Released,Asterix at the Olympic Games,5.0,471,Clovis Cornillac | Gérard Depardieu,2008,Thomas Langmann,Pierre Grunstein
4,0,Comedy | Fantasy | Family,10588,Comedy | Fantasy | Family,en,18.251129,Imagine Entertainment | DreamWorks SKG | Unive...,United States of America,0,82.0,Released,The Cat in the Hat,4.9,366,Mike Myers | Dakota Fanning,2003,Bo Welch,Eric McLeod


In [3]:
df['decade'] = (df['year'] // 10) * 10 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3790 entries, 0 to 3789
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                3790 non-null   int64  
 1   genres                3790 non-null   object 
 2   id                    3790 non-null   int64  
 3   keywords              3790 non-null   object 
 4   original_language     3790 non-null   object 
 5   popularity            3790 non-null   float64
 6   production_companies  3790 non-null   object 
 7   production_countries  3790 non-null   object 
 8   revenue               3790 non-null   int64  
 9   runtime               3790 non-null   float64
 10  status                3790 non-null   object 
 11  title                 3790 non-null   object 
 12  vote_average          3790 non-null   float64
 13  vote_count            3790 non-null   int64  
 14  cast                  3774 non-null   object 
 15  year                 

In [4]:
# Split the genres string into a list, filter out empty strings and pipe characters, then count
all_genres = [genre for genres in df['genres'].str.split(' | ') 
              for genre in genres 
              if genre.strip() and genre != '|']
genre_counts = pd.Series(all_genres).value_counts()
unique_genres = sorted(set(all_genres))
print(genre_counts)

Drama          1942
Comedy         1446
Thriller       1021
Action          786
Romance         775
Crime           580
Adventure       472
Horror          454
Family          350
Science         342
Fiction         342
Mystery         281
Fantasy         264
Music           162
History         138
Animation       125
War              97
Documentary      94
Western          59
Foreign          31
Unknown           8
TV                5
Movie             5
Name: count, dtype: int64


### Adjusted Revenue and Budget

In [5]:
import pandas_datareader.data as web

# Download monthly CPI (All Urban Consumers) series
cpi = web.DataReader(
    'CPIAUCSL',           # CPI for All Urban Consumers: All Items
    data_source='fred',
    start='1920-01-01',   # adjust to your earliest movie year
    end='2023-12-31'
)

#  Convert to annual averages
cpi['Year'] = cpi.index.year
cpi_annual = (
    cpi
    .groupby('Year')['CPIAUCSL']
    .mean()
    .reset_index()
    .rename(columns={'CPIAUCSL':'CPI'})
)

# 3) Save to disk
cpi_annual.to_csv('cpi.csv', index=False)

print(f"Saved CPI file with {len(cpi_annual)} rows, from {cpi_annual['Year'].min()} to {cpi_annual['Year'].max()}")

Saved CPI file with 77 rows, from 1947 to 2023


In [6]:
# Load CPI data and compute adjustment factors
cpi = pd.read_csv('cpi.csv')  # must have columns: Year,CPI
cpi.rename(columns={'Year':'year','CPI':'cpi'}, inplace=True)
base_cpi = cpi.loc[cpi['year']==2023, 'cpi'].iloc[0]
cpi['adj_factor'] = base_cpi / cpi['cpi']

In [7]:
# 6) Merge CPI factors and compute adjusted financials
infl = df[(df['budget'] > 0) & (df['revenue'] > 0)].copy()
infl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2398 entries, 1 to 3785
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                2398 non-null   int64  
 1   genres                2398 non-null   object 
 2   id                    2398 non-null   int64  
 3   keywords              2398 non-null   object 
 4   original_language     2398 non-null   object 
 5   popularity            2398 non-null   float64
 6   production_companies  2398 non-null   object 
 7   production_countries  2398 non-null   object 
 8   revenue               2398 non-null   int64  
 9   runtime               2398 non-null   float64
 10  status                2398 non-null   object 
 11  title                 2398 non-null   object 
 12  vote_average          2398 non-null   float64
 13  vote_count            2398 non-null   int64  
 14  cast                  2396 non-null   object 
 15  year                  2398

In [8]:
infl = infl.merge(cpi[['year','adj_factor']], on='year', how='left')
infl['budget_adj']  = infl['budget']  * infl['adj_factor']
infl['revenue_adj'] = infl['revenue'] * infl['adj_factor']
infl['roi_adj']     = infl['revenue_adj'] / infl['budget_adj']

In [9]:
infl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2398 entries, 0 to 2397
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                2398 non-null   int64  
 1   genres                2398 non-null   object 
 2   id                    2398 non-null   int64  
 3   keywords              2398 non-null   object 
 4   original_language     2398 non-null   object 
 5   popularity            2398 non-null   float64
 6   production_companies  2398 non-null   object 
 7   production_countries  2398 non-null   object 
 8   revenue               2398 non-null   int64  
 9   runtime               2398 non-null   float64
 10  status                2398 non-null   object 
 11  title                 2398 non-null   object 
 12  vote_average          2398 non-null   float64
 13  vote_count            2398 non-null   int64  
 14  cast                  2396 non-null   object 
 15  year                 

In [10]:
# 8) Drop any rows missing raw budget or revenue or adjustment
infl = infl.dropna(subset=['budget','revenue','adj_factor'])

In [11]:
infl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2371 entries, 0 to 2397
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                2371 non-null   int64  
 1   genres                2371 non-null   object 
 2   id                    2371 non-null   int64  
 3   keywords              2371 non-null   object 
 4   original_language     2371 non-null   object 
 5   popularity            2371 non-null   float64
 6   production_companies  2371 non-null   object 
 7   production_countries  2371 non-null   object 
 8   revenue               2371 non-null   int64  
 9   runtime               2371 non-null   float64
 10  status                2371 non-null   object 
 11  title                 2371 non-null   object 
 12  vote_average          2371 non-null   float64
 13  vote_count            2371 non-null   int64  
 14  cast                  2369 non-null   object 
 15  year                  2371

In [12]:
import plotly.express as px
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [13]:
app = dash.Dash(__name__)

In [14]:
# Exploding genres into individual genres
df_expl = (
    df
    # .assign(decade = (df['year'] // 10) * 10)  # Compute the decade
    .assign(genres=df['genres'].str.split(r' \| '))  # Split the genre by the pipe symbol
    .explode('genres')  # Explode the genres into separate rows
)

# Add necessary columns for ranking (popularity, vote_average, revenue)
df_expl['popularity'] = df['popularity']
df_expl['vote_average'] = df['vote_average']
df_expl['revenue'] = df['revenue']
# Count and rank genres per decade
df_counts = (
    df_expl
    .groupby(['decade', 'genres'])
    .size()
    .reset_index(name='count')  
)

print(df_counts.columns)
df_expl_agg = (
    df_expl
    .groupby(['decade', 'genres'], as_index=False)
    .agg({
        'popularity': 'mean',   # Mean of popularity for each genre per decade
        'vote_average': 'mean', # Mean of vote_average for each genre per decade
        'revenue': 'sum'        # Sum of revenue for each genre per decade
    })
)

# Merge the count column from df_counts with the aggregated features
df_counts_aggregated = df_counts.merge(df_expl_agg, on=['decade', 'genres'], how='left')

Index(['decade', 'genres', 'count'], dtype='object')


### App Layout

In [15]:
app.layout = html.Div([
    # Sidebar
    html.Div([
        dcc.Dropdown(
            id='top-n-dropdown',
            options=[{'label': str(i), 'value': i} for i in range(2, 6)],
            value=2,
            style={'width': '97.5%', 'margin': '2.5% auto'}
        ),
        dcc.Dropdown(
            id='rank-feature-dropdown',
            options=[
                {'label': 'Number of Movies', 'value': 'count'},
                {'label': 'Popularity', 'value': 'popularity'},
                {'label': 'Vote Average', 'value': 'vote_average'},
                {'label': 'Revenue', 'value': 'revenue'}
            ],
            value='count',
            style={'width': '97.5%', 'margin': '2.5% auto'}
        ),
        dcc.Dropdown(
            id='genre-dropdown',
            options=[{'label': genre, 'value': genre} for genre in unique_genres],
            style={'width': '97.5%', 'margin': '2.5% auto'}
        ),
        dcc.Dropdown(
            id='decade-dropdown',
            options=[{'label': decade, 'value': decade} for decade in sorted(infl['decade'].unique())],
            placeholder="Select Decade",
            value=None,
            style={'width': '97.5%', 'margin': '2.5% auto'}
        )
    ], style={
        'position': 'fixed',
        'top': 0,
        'left': 0,
        'width': '15%',
        'height': '100vh',
        'padding': '1%',
        'backgroundColor': '#f8f9fa',
        'overflowY': 'auto'
    }),

    # Main Content
    html.Div([
        # Row 1: Bump & Line Charts
        html.Div([
            html.Div(dcc.Graph(id='bump-chart'), style={'width': '50%', 'display': 'inline-block'}),
            html.Div(dcc.Graph(id='line-chart'), style={'width': '50%', 'display': 'inline-block'})
        ], style={'display': 'flex', 'width': '100%'}),

        # Row 2: Scatter and Violin stacked
        html.Div([
            html.Div(dcc.Graph(id='scatter'), style={'width': '50%', 'marginBottom': '0.5%'}),
            html.Div(dcc.Graph(id='violin'), style={'width': '50%'})
        ], style={'display': 'flex', 'width': '100%'})
        
    ], style={
        'marginLeft': '16%',  # account for sidebar
        'padding': '0.5%'
    })
])


In [16]:
# Now, handle the rank calculation based on the selected ranking feature
@app.callback(
    Output('bump-chart', 'figure'),
    [Input('top-n-dropdown', 'value'),
     Input('rank-feature-dropdown', 'value')]
)
def update_bump_chart(top_n, rank_feature):
    df_counts_aggregated['rank'] = df_counts_aggregated.groupby('decade')[rank_feature]\
                                                      .rank(method='first', ascending=False)

    # Filter data to keep only top N genres per decade
    bump = df_counts_aggregated[df_counts_aggregated['rank'] <= top_n]

    genre_list = genre_counts.index.tolist()

    # Step 1: Generate a list of dark colors (using matplotlib's dark colormap or manual color selection)
    dark_colors = ['#8B4513', '#3cb44b', '#FF0000', '#4363d8', '#f58231', '#911eb4', '#00BFFF', '#f032e6', '#bcf60c', '#C71585', 
                   '#008080', '#e6beff', '#9a6324', '#800000', '#D5006D', '#2E8B57', '#1E90FF', '#000075', '#808080', '#000000',
                  '#FF00FF','#FF4500','#00FF7F']
    
    # Step 2: Ensure that we have enough colors for all genres
    # If there are more genres than the dark colors, repeat the dark color palette
    if len(genre_list) > len(dark_colors):
        dark_colors = (dark_colors * (len(genre_list) // len(dark_colors) + 1))[:len(genre_list)]

    # Step 3: Map genres to colors
    genre_color_mapping = dict(zip(genre_list, dark_colors))

    df_expl['color'] = df_expl['genres'].map(genre_color_mapping)
    # Create the bump chart using Plotly
    fig = px.line(
        bump,
        x='decade', y='rank',
        color='genres',  # Ensure genres are being mapped to distinct colors
        markers=True,
        line_shape='spline',
        hover_data=['count'],
        title=f'Top {top_n} Genres per Decade (Bump Chart) - Ranked by {rank_feature}',
        labels={'decade': 'Decade', 'rank': 'Rank', 'genres': 'Genre'},
        color_discrete_map=genre_color_mapping,
    )

    # Reverse the y-axis to have rank 1 at the top
    fig.update_yaxes(autorange='reversed', dtick=1)
    fig.update_xaxes(dtick=10)

    return fig


In [17]:
@app.callback(
    Output('line-chart', 'figure'),
    [Input('genre-dropdown', 'value')]
)
def temporal_trends(selected_genre):
    working_df = df.copy()

    # Filter by genre only if a genre is selected
    if selected_genre:
        working_df = working_df[working_df['genres'].str.contains(selected_genre, na=False)]
    
    # Count movies per year
    movies_per_year = working_df['decade'].value_counts().sort_index()
    trend_df = movies_per_year.reset_index()
    trend_df.columns = ['Decade', 'Number of Movies']

    # Plot with Plotly
    fig = px.line(trend_df, x='Decade', y='Number of Movies',
                  title=f"Movies Released per Decade{' - ' + selected_genre if selected_genre else ''}",
                  labels={'decade': 'decade', 'Number of Movies': 'Number of Movies'})
    fig.update_layout(xaxis_title="Decade", yaxis_title="Number of Movies")  # Show every year on x-axis if needed
    return fig


In [18]:
@app.callback(
    Output('scatter', 'figure'),
    [Input('genre-dropdown', 'value'),
     Input('decade-dropdown', 'value')]
)
def revnue_budget(selected_genre, selected_decade):
    working_df = infl.copy()

    # Filter by genre only if a genre is selected
    if selected_genre:
        working_df = working_df[working_df['genres'].str.contains(selected_genre, na=False)]
    if selected_decade is not None:
        working_df = working_df[working_df['decade'] == selected_decade]
        
    fig = px.scatter(
        working_df,
        x='budget_adj',
        y='revenue_adj',
        hover_data=['title', 'year', 'genres'],  # use 'year'
        log_x=True, log_y=True, opacity=0.5,
        title='Inflation-Adjusted Budget vs Revenue',
        labels={
            'budget_adj': 'Budget (2023 USD)',
            'revenue_adj': 'Revenue (2023 USD)',
            'year': 'Release Year'
        }
    )
    return fig

In [19]:
@app.callback(
    Output('violin', 'figure'),
    [Input('genre-dropdown', 'value')]
)

def runtime(selected_genre):
    working_df = df.copy()
    if selected_genre:
        working_df = working_df[working_df['genres'].str.contains(selected_genre, na=False)]
    
    fig = px.violin(
        working_df,
        x='decade',
        y='runtime',
        box=True,            # add boxplot inside
        points=False,        # hide default px points
        labels={'decade':'Decade','runtime':'Runtime (min)'},
        color_discrete_sequence=px.colors.qualitative.Safe
    )
    
    fig.update_traces(
        selector={'type':'violin'},
        line_color='blue',
        line_width=0.5,
        points='all',        
        jitter=0.3,          
        marker=dict(size=4, opacity=0.3)
    )
    
    fig.update_layout(
        title='Runtime Distribution by Decade (Voilin + Beeswarm, zero‐runtimes dropped)',
        template='plotly_white'
    )
    
    return fig

In [20]:
# Run the app
if __name__ == '__main__':
    app.run(debug=True)

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[16], line 8, in update_bump_chart(top_n=2, rank_feature=None)
      2 @app.callback(
      3     Output('bump-chart', 'figure'),
      4     [Input('top-n-dropdown', 'value'),
      5      Input('rank-feature-dropdown', 'value')]
      6 )
      7 def update_bump_chart(top_n, rank_feature):
----> 8     df_counts_aggregated['rank'] = df_counts_aggregated.groupby('decade')[rank_feature]\
        df_counts_aggregated =      decade           genres  count  popularity  vote_average     revenue  \
0      1920            Drama      4    8.982580      6.900000    27008422   
1      1920            Music      1    0.968865      5.000000     4358000   
2      1920          Romance      3    1.192931      6.533333    26358000   
3      1920  Science Fiction      1   32.351527      8.000000      650422   
4      1920         Thriller      1